# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [48]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [121]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpa3nv60jw\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet


C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp3mttvvzx\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet


C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


...

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpl9jtzvdv\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=11\part-00021-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2591-1.c000.snappy.parquet


C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpp3040232\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=12\part-00116-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2686-1.c000.snappy.parquet


vendorID  lpepPickupDatetime lpepDropoffDatetime  passengerCount  \
471305         1 2016-01-08 19:18:54 2016-01-08 19:34:01               1   
809788         1 2016-01-20 07:06:26 2016-01-20 07:21:21               1   
914750         1 2016-01-05 18:30:36 2016-01-05 19:26:05               1   
614014         2 2016-01-17 12:24:05 2016-01-17 12:32:46               1   
643172         1 2016-01-30 19:48:41 2016-01-30 19:58:57               2   
...          ...                 ...                 ...             ...   
719297         2 2016-12-23 19:21:18 2016-12-23 19:34:57               1   
56239          2 2016-12-05 16:32:51 2016-12-05 16:37:32               1   
262873         2 2016-12-07 13:16:07 2016-12-07 13:32:56               1   
914017         2 2016-12-14 12:23:23 2016-12-14 12:28:55               1   
408309         2 2016-12-17 09:20:55 2016-12-17 09:38:38               1   

        tripDistance puLocationId doLocationId  pickupLongitude  \
471305          1.40         None         None       -73.992409   
809788          4.30         None         None       -73.984230   
914750         13.00         None         None       -73.962410   
614014          1.95         None         None       -73.944923   
643172          1.80         None         None       -73.949043   
...              ...          ...          ...              ...   
719297          4.41           40          125              NaN   
56239           0.98           41           42              NaN   
262873          3.68           49           87              NaN   
914017          1.17          147          159              NaN   
408309          3.02           42           69              NaN   

        pickupLatitude  dropoffLongitude  ...  paymentType  fareAmount extra  \
471305       40.689312        -73.970985  ...            2        10.5   1.0   
809788       40.666580        -74.013489  ...            2        15.5   0.0   
914750       40.677422        -73.765015  ...            1        45.0   1.0   
614014       40.670570        -73.974556  ...            1         8.5   0.0   
643172       40.785198        -73.965408  ...            1         9.0   0.0   
...                ...               ...  ...          ...         ...   ...   
719297             NaN               NaN  ...            1        15.0   1.0   
56239              NaN               NaN  ...            1         5.5   1.0   
262873             NaN               NaN  ...            1        14.5   0.0   
914017             NaN               NaN  ...            2         6.5   0.0   
408309             NaN               NaN  ...            2        13.5   0.0   

        mtaTax  improvementSurcharge  tipAmount  tollsAmount ehailFee  \
471305     0.5                   0.3       0.00         0.00      NaN   
809788     0.5                   0.3       0.00         5.54      NaN   
914750     0.5                   0.3       0.00         0.00      NaN   
614014     0.5                   0.3       1.86         0.00      NaN   
643172     0.5                   0.3       1.95         0.00      NaN   
...        ...                   ...        ...          ...      ...   
719297     0.5                   0.3       4.47         5.54      NaN   
56239      0.5                   0.3       1.00         0.00      NaN   
262873     0.5                   0.3       0.00         0.00      NaN   
914017     0.5                   0.3       0.00         0.00      NaN   
408309     0.5                   0.3       0.00         0.00      NaN   

        totalAmount  tripType  
471305        12.30       1.0  
809788        21.84       1.0  
914750        46.80       1.0  
614014        11.16       1.0  
643172        11.75       1.0  
...             ...       ...  
719297        26.81       1.0  
56239          8.30       1.0  
262873        15.30       1.0  
914017         7.30       1.0  
408309        14.30       1.0  

[24000 rows x 23 columns]

Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [123]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

vendorID  lpepPickupDatetime lpepDropoffDatetime  passengerCount  \
471305         1 2016-01-08 19:18:54 2016-01-08 19:34:01               1   
809788         1 2016-01-20 07:06:26 2016-01-20 07:21:21               1   
914750         1 2016-01-05 18:30:36 2016-01-05 19:26:05               1   
614014         2 2016-01-17 12:24:05 2016-01-17 12:32:46               1   
643172         1 2016-01-30 19:48:41 2016-01-30 19:58:57               2   
...          ...                 ...                 ...             ...   
719297         2 2016-12-23 19:21:18 2016-12-23 19:34:57               1   
56239          2 2016-12-05 16:32:51 2016-12-05 16:37:32               1   
262873         2 2016-12-07 13:16:07 2016-12-07 13:32:56               1   
914017         2 2016-12-14 12:23:23 2016-12-14 12:28:55               1   
408309         2 2016-12-17 09:20:55 2016-12-17 09:38:38               1   

        tripDistance puLocationId doLocationId  pickupLongitude  \
471305          1.40         None         None       -73.992409   
809788          4.30         None         None       -73.984230   
914750         13.00         None         None       -73.962410   
614014          1.95         None         None       -73.944923   
643172          1.80         None         None       -73.949043   
...              ...          ...          ...              ...   
719297          4.41           40          125              NaN   
56239           0.98           41           42              NaN   
262873          3.68           49           87              NaN   
914017          1.17          147          159              NaN   
408309          3.02           42           69              NaN   

        pickupLatitude  dropoffLongitude  ...  tripType  month_num  \
471305       40.689312        -73.970985  ...       1.0          1   
809788       40.666580        -74.013489  ...       1.0          1   
914750       40.677422        -73.765015  ...       1.0          1   
614014       40.670570        -73.974556  ...       1.0          1   
643172       40.785198        -73.965408  ...       1.0          1   
...                ...               ...  ...       ...        ...   
719297             NaN               NaN  ...       1.0         12   
56239              NaN               NaN  ...       1.0         12   
262873             NaN               NaN  ...       1.0         12   
914017             NaN               NaN  ...       1.0         12   
408309             NaN               NaN  ...       1.0         12   

       day_of_month  day_of_week  hour_of_day  country_code        hr_sin  \
471305            8            4           19            US -9.659258e-01   
809788           20            2            7            US  9.659258e-01   
914750            5            1           18            US -1.000000e+00   
614014           17            6           12            US  1.224647e-16   
643172           30            5           19            US -9.659258e-01   
...             ...          ...          ...           ...           ...   
719297           23            4           19            US -9.659258e-01   
56239             5            0           16            US -8.660254e-01   
262873            7            2           13            US -2.588190e-01   
914017           14            2           12            US  1.224647e-16   
408309           17            5            9            US  7.071068e-01   

              hr_cos    dy_sin    dy_cos  
471305  2.588190e-01 -0.433884 -0.900969  
809788 -2.588190e-01  0.974928 -0.222521  
914750 -1.836970e-16  0.781831  0.623490  
614014 -1.000000e+00 -0.781831  0.623490  
643172  2.588190e-01 -0.974928 -0.222521  
...              ...       ...       ...  
719297  2.588190e-01 -0.433884 -0.900969  
56239  -5.000000e-01  0.000000  1.000000  
262873 -9.659258e-01  0.974928 -0.222521  
914017 -1.000000e+00  0.974928 -0.222521  
408309 -7.071068e-01 -0.974928 -0.222521  

[24000 

Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [124]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

vendorID  lpepPickupDatetime  passengerCount  tripDistance  \
471305         1 2016-01-08 19:18:54               1          1.40   
809788         1 2016-01-20 07:06:26               1          4.30   
914750         1 2016-01-05 18:30:36               1         13.00   
614014         2 2016-01-17 12:24:05               1          1.95   
643172         1 2016-01-30 19:48:41               2          1.80   

        pickupLongitude  pickupLatitude  dropoffLongitude  dropoffLatitude  \
471305       -73.992409       40.689312        -73.970985        40.684914   
809788       -73.984230       40.666580        -74.013489        40.715286   
914750       -73.962410       40.677422        -73.765015        40.684956   
614014       -73.944923       40.670570        -73.974556        40.680145   
643172       -73.949043       40.785198        -73.965408        40.801041   

        totalAmount  month_num  day_of_month  day_of_week  hour_of_day  \
471305        12.30          1             8            4           19   
809788        21.84          1            20            2            7   
914750        46.80          1             5            1           18   
614014        11.16          1            17            6           12   
643172        11.75          1            30            5           19   

       country_code        hr_sin        hr_cos    dy_sin    dy_cos   datetime  
471305           US -9.659258e-01  2.588190e-01 -0.433884 -0.900969 2016-01-08  
809788           US  9.659258e-01 -2.588190e-01  0.974928 -0.222521 2016-01-20  
914750           US -1.000000e+00 -1.836970e-16  0.781831  0.623490 2016-01-05  
614014           US  1.224647e-16 -1.000000e+00 -0.781831  0.623490 2016-01-17  
643172           US -9.659258e-01  2.588190e-01 -0.974928 -0.222521 2016-01-30

### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [127]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpcxz6548w\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


countryOrRegion                 holidayName        normalizeHolidayName  \
19375       Argentina  Año Nuevo [New Year's Day]  Año Nuevo [New Year's Day]   
19376       Australia              New Year's Day              New Year's Day   
19377         Austria                     Neujahr                     Neujahr   
19378         Belarus                   Новый год                   Новый год   
19379         Belgium               Nieuwjaarsdag               Nieuwjaarsdag   

      isPaidTimeOff countryRegionCode       date  
19375          None                AR 2008-01-01  
19376          None                AU 2008-01-01  
19377          None                AT 2008-01-01  
19378          None                BY 2008-01-01  
19379          None                BE 2008-01-01

Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [128]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

vendorID  lpepPickupDatetime  passengerCount  tripDistance  \
28            2 2016-01-01 02:18:07               1          4.86   
29            2 2016-01-18 12:53:10               1          3.46   
33            2 2016-01-18 10:42:37               1          1.24   
38            2 2016-01-01 18:19:18               6          3.41   
39            2 2016-01-01 23:54:14               1          2.85   
...         ...                 ...             ...           ...   
23968         2 2016-12-25 09:46:27               2          0.85   
23969         1 2016-12-26 22:30:33               2          1.40   
23975         2 2016-12-25 04:25:11               1          0.59   
23976         2 2016-12-26 10:24:07               1          1.71   
23979         1 2016-12-26 21:08:39               1          2.30   

       pickupLongitude  pickupLatitude  dropoffLongitude  dropoffLatitude  \
28          -73.958382       40.717327        -73.909164        40.758812   
29          -73.939217       40.805515        -73.955994        40.764565   
33          -73.948921       40.796955        -73.937119        40.801991   
38          -73.928238       40.755928        -73.892242        40.770615   
39          -73.807091       40.699322        -73.812218        40.673378   
...                ...             ...               ...              ...   
23968              NaN             NaN               NaN              NaN   
23969              NaN             NaN               NaN              NaN   
23975              NaN             NaN               NaN              NaN   
23976              NaN             NaN               NaN              NaN   
23979              NaN             NaN               NaN              NaN   

       totalAmount  month_num  ...  day_of_week  hour_of_day  country_code  \
28           25.56          1  ...            4            2            US   
29           15.30          1  ...            0           12            US   
33            8.97          1  ...            0           10            US   
38           14.80          1  ...            4           18            US   
39           12.55          1  ...            4           23            US   
...            ...        ...  ...          ...          ...           ...   
23968         5.30         12  ...            6            9            US   
23969         8.30         12  ...            0           22            US   
23975         5.80         12  ...            6            4            US   
23976        10.56         12  ...            0           10            US   
23979        11.80         12  ...            0           21            US   

             hr_sin        hr_cos    dy_sin    dy_cos   datetime  \
28     5.000000e-01  8.660254e-01 -0.433884 -0.900969 2016-01-01   
29     1.224647e-16 -1.000000e+00  0.000000  1.000000 2016-01-18   
33     5.000000e-01 -8.660254e-01  0.000000  1.000000 2016-01-18   
38    -1.000000e+00 -1.836970e-16 -0.433884 -0.900969 2016-01-01   
39    -2.588190e-01  9.659258e-01 -0.433884 -0.900969 2016-01-01   
...             ...           ...       ...       ...        ...   
23968  7.071068e-01 -7.071068e-01 -0.781831  0.623490 2016-12-25   
23969 -5.000000e-01  8.660254e-01  0.000000  1.000000 2016-12-26   
23975  8.660254e-01  5.000000e-01 -0.781831  0.623490 2016-12-25   
23976  5.000000e-01 -8.660254e-01  0.000000  1.000000 2016-12-26   
23979 -7.071068e-01  7.071068e-01  0.000000  1.000000 2016-12-26   

             normalizeHolidayName isPaidTimeOff  
28                 New Year's Day          True  
29     Martin Luther King Jr. Day          None  
33     Martin Luther King Jr. Day          None  
38                 New Year's Day          True  
39                 New Year's Day          True  
...                           ...           ...  
23968               Christmas Day          True  
23969               Christmas Day          True  
23975               Christmas Day          True  

### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [69]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp7snwb4fh\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp7snwb4fh\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp7snwb4fh\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp7snwb4fh\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-44

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpmf1zb6v_\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=2\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-2.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpmf1zb6v_\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=2\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-2.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpmf1zb6v_\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=2\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-2.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpmf1zb6v_\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=2\part-00003-tid-738723883827836859-85e0759b-51fa-44

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


...

[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpuv8p344e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=10\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-10.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpuv8p344e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=10\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-10.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpuv8p344e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=10\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-10.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpuv8p344e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=10\part-00003-tid-738723883827836859-85e0759b-

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpitlt3sej\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=11\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-11.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpitlt3sej\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=11\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-11.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpitlt3sej\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=11\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-11.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmpitlt3sej\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=11\part-00003-tid-738723883827836859-85e0759b-

C:\Users\nserafino\AppData\Roaming\Python\Python38\site-packages\azureml\opendatasets\dataaccess\_blob_accessor.py:519: Warning: Please install azureml-dataset-runtimeusing pip install azureml-dataset-runtime
  warnings.warn(


[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp6_8bqc_w\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=12\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-12.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp6_8bqc_w\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=12\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-12.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp6_8bqc_w\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=12\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-12.c000.snappy.parquet
[Info] read from C:\Users\NSERAF~1\AppData\Local\Temp\tmp6_8bqc_w\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=12\part-00003-tid-738723883827836859-85e0759b-

In [130]:
weather_df

precipDepth  precipTime   wban  temperature  snowDepth    usaf  \
204647           NaN         NaN  14732          2.8        NaN  725030   
204670           0.0         1.0  14732         -4.4        NaN  725030   
204694           0.0         1.0  14732          5.0        NaN  725030   
204701           0.0         1.0  14732         -1.1        NaN  725030   
204715           0.0         1.0  14732          4.4        NaN  725030   
...              ...         ...    ...          ...        ...     ...   
1248471          0.0         1.0  94728          4.4        NaN  725053   
1248555          0.0         1.0  94728          5.0        NaN  725053   
1248580          NaN         NaN  94728          3.9        NaN  725053   
1248597          0.0         1.0  94728          7.8        NaN  725053   
1248600          5.0         1.0  94728         -2.8        NaN  725053   

         longitude   datetime  latitude  
204647     -73.867 2016-01-02    40.783  
204670     -73.880 2016-01-22    40.779  
204694     -73.880 2016-01-08    40.779  
204701     -73.880 2016-01-04    40.779  
204715     -73.880 2016-01-01    40.779  
...            ...        ...       ...  
1248471    -73.967 2016-12-23    40.789  
1248555    -73.967 2016-12-12    40.789  
1248580    -73.967 2016-12-18    40.789  
1248597    -73.967 2016-12-25    40.789  
1248600    -73.967 2016-12-17    40.789  

[55683 rows x 9 columns]

Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [153]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

precipTime  temperature  precipDepth
datetime                                        
2016-01-01         1.0     5.197345          0.0
2016-01-02         1.0     2.567857          0.0
2016-01-03         1.0     3.846429          0.0
2016-01-04         1.0     0.123894          0.0
2016-01-05         6.0    -7.206250          0.0
2016-01-06         6.0    -0.896396          0.0
2016-01-07         6.0     3.180645          0.0
2016-01-08         1.0     4.384091          0.0
2016-01-09         6.0     6.710274          3.0
2016-01-10        24.0    10.943655        254.0

Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [131]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

vendorID  passengerCount  tripDistance  pickupLongitude  \
count  24000.000000    24000.000000  24000.000000     12000.000000   
mean       1.792000        1.354500      2.768621       -73.819910   
std        0.405885        1.014909      2.844013         2.940137   
min        1.000000        0.000000      0.000000       -74.321251   
25%        2.000000        1.000000      1.020000       -73.961100   
50%        2.000000        1.000000      1.810000       -73.946396   
75%        2.000000        1.000000      3.500000       -73.918844   
max        2.000000        8.000000     94.900000         0.000000   

       pickupLatitude  dropoffLongitude  dropoffLatitude   totalAmount  \
count    12000.000000      12000.000000     12000.000000  24000.000000   
mean        40.683108        -73.818512        40.681168     14.526059   
std          1.621175          2.940219         1.621136     11.141177   
min          0.000000        -74.581146         0.000000    -80.000000   
25%         40.694138        -73.968309        40.695732      7.800000   
50%         40.746460        -73.945339        40.747179     11.300000   
75%         40.801885        -73.912537        40.791529     17.480000   
max         40.909771          0.000000        40.962906    534.300000   

          month_num  day_of_month  day_of_week   hour_of_day        hr_sin  \
count  24000.000000  24000.000000  24000.00000  24000.000000  24000.000000   
mean       6.500000     15.033167      3.22125     13.614042     -0.242673   
std        3.452124      8.440722      1.95624      6.661438      0.666527   
min        1.000000      1.000000      0.00000      0.000000     -1.000000   
25%        3.750000      8.000000      2.00000      9.000000     -0.866025   
50%        6.500000     15.000000      3.00000     15.000000     -0.500000   
75%        9.250000     22.000000      5.00000     19.000000      0.258819   
max       12.000000     30.000000      6.00000     23.000000      1.000000   

             hr_cos        dy_sin        dy_cos    snowDepth    precipTime  \
count  2.400000e+04  24000.000000  24000.000000  1661.000000  24000.000000   
mean  -2.294540e-02     -0.076013     -0.055439    11.832133     13.333333   
std    7.045314e-01      0.714573      0.693235    15.604274     10.330900   
min   -1.000000e+00     -0.974928     -0.900969     3.000000      1.000000   
25%   -7.071068e-01     -0.781831     -0.900969     3.000000      6.000000   
50%   -1.836970e-16      0.000000     -0.222521     4.428571      6.000000   
75%    7.071068e-01      0.781831      0.623490    12.722222     24.000000   
max    1.000000e+00      0.974928      1.000000    67.090909     24.000000   

        temperature   precipDepth  
count  24000.000000  24000.000000  
mean      13.896879   1050.780125  
std        9.458729   2805.910927  
min      -13.379464      0.000000  
25%        6.620773      0.000000  
50%       13.125893     10.000000  
75%       22.944737    127.000000  
max       31.303665   9999.000000

From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [132]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [155]:
final_df.describe()

passengerCount  tripDistance  pickupLongitude  pickupLatitude  \
count    11758.000000  11758.000000     11758.000000    11758.000000   
mean         1.357289      2.826601       -73.937390       40.747138   
std          1.018249      2.752486         0.041107        0.056387   
min          1.000000      0.010000       -74.087898       40.575096   
25%          1.000000      1.070000       -73.961248       40.694242   
50%          1.000000      1.870000       -73.946831       40.746342   
75%          1.000000      3.580000       -73.919100       40.801531   
max          8.000000     36.300000       -73.736725       40.879841   

       dropoffLongitude  dropoffLatitude   totalAmount   day_of_week  \
count      11758.000000     11758.000000  11758.000000  11758.000000   
mean         -73.898243        40.724478     14.557613      3.220276   
std            1.670565         0.922020      9.968744      1.965153   
min          -74.183289         0.000000      1.000000      0.000000   
25%          -73.968748        40.695820      8.150000      2.000000   
50%          -73.945671        40.746984     11.300000      3.000000   
75%          -73.913416        40.790767     17.760000      5.000000   
max            0.000000        40.934597    154.060000      6.000000   

        hour_of_day        hr_sin        hr_cos        dy_sin        dy_cos  \
count  11758.000000  11758.000000  1.175800e+04  11758.000000  11758.000000   
mean      13.556727     -0.239765 -2.278258e-02     -0.075822     -0.048049   
std        6.674625      0.670335  7.019437e-01      0.716176      0.692184   
min        0.000000     -1.000000 -1.000000e+00     -0.974928     -0.900969   
25%        9.000000     -0.866025 -7.071068e-01     -0.781831     -0.900969   
50%       15.000000     -0.500000 -1.836970e-16      0.000000     -0.222521   
75%       19.000000      0.258819  7.071068e-01      0.781831      0.623490   
max       23.000000      1.000000  1.000000e+00      0.974928      1.000000   

         snowDepth    precipTime   temperature   precipDepth  
count  1495.000000  11758.000000  11758.000000  11758.000000  
mean     12.633662     11.928219     10.265271    205.724018  
std      16.192875     10.128046      8.493089   1275.548381  
min       3.000000      1.000000    -13.379464      0.000000  
25%       3.000000      1.000000      3.566372      0.000000  
50%       4.428571      6.000000     10.130357      3.000000  
75%      15.647059     24.000000     16.966923     41.000000  
max      67.090909     24.000000     26.524107   9999.000000

## Train a model

The data is ready to train a machine learning model.

In [156]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [135]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    model = LinearRegression()
  elif algo_name == 'random_forest':
    model = RandomForestRegressor()
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [136]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [157]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression", "random_forest"]:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.8780540715956401
MAPE: 0.15410376701239029
RMSE: 3.3947219965481423

random_forest
R2: 0.8625262581779325
MAPE: 0.15554265051292004
RMSE: 3.6043789033118308

